In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../inpreprocessing ```Age```put/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# make sure to turn on the Interenet switch in the right side menu
! pip install pyspark

     |████████████████████████████████| 281.5MB 32kB/s s eta 0:00:01     |██████████████████▋             | 164.1MB 56.2MB/s eta 0:00:03
     |████████████████████████████████| 204kB 35.3MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.2.4-py2.py3-none-any.whl size=282040934 sha256=37b0cb7095f1e2e0f685cc65c0c08933b544eefbb8b6cc633ce0f575c8175683
  Stored in directory: /root/.cache/pip/wheels/89/d7/05/fbb69079403a2fc4f827f02c7557f1cb4dbd9d122af547f61a
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, StringType, StructType, StructField
from pyspark.ml.feature import StringIndexer, VectorAssembler, QuantileDiscretizer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark import SparkContext
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [4]:
# build spark session
spark = SparkSession.builder.appName("Titanic Analysis").getOrCreate()

/opt/conda/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


## 5. Data Loading

In [5]:
train = spark.read.csv("/kaggle/input/titanic/train.csv", header="true", inferSchema="true")
test = spark.read.csv("/kaggle/input/titanic/train.csv", header="true", inferSchema="true")

In [6]:
titanic_df = spark.read.csv("/kaggle/input/titanic/train.csv", header="true", inferSchema="true")

In [7]:
# Display the schema to understand the structure
titanic_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [8]:
type(titanic_df)

pyspark.sql.dataframe.DataFrame

In [9]:
titanic_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

### Performing Basic Operations in Spark

In [10]:
survival_rate_by_gender = titanic_df.groupBy("Sex").agg({"Survived": "mean"})
survival_rate_by_gender.show()

+------+-------------------+
|   Sex|      avg(Survived)|
+------+-------------------+
|female| 0.7420382165605095|
|  male|0.18890814558058924|
+------+-------------------+



In [11]:
# Sort the Titanic DataFrame by Fare in descending order using sort()
sorted_df = titanic_df.sort(titanic_df.Fare.desc())
sorted_df.select("PassengerId", "Fare", "Name").show(5)

+-----------+--------+--------------------+
|PassengerId|    Fare|                Name|
+-----------+--------+--------------------+
|        680|512.3292|Cardeza, Mr. Thom...|
|        259|512.3292|    Ward, Miss. Anna|
|        738|512.3292|Lesurer, Mr. Gust...|
|         89|   263.0|Fortune, Miss. Ma...|
|         28|   263.0|Fortune, Mr. Char...|
+-----------+--------+--------------------+
only showing top 5 rows



In [12]:
sorted_by_fare = titanic_df.orderBy("Fare", ascending=False)
sorted_by_fare.select("PassengerId", "Fare", "Name").show(5)

+-----------+--------+--------------------+
|PassengerId|    Fare|                Name|
+-----------+--------+--------------------+
|        680|512.3292|Cardeza, Mr. Thom...|
|        259|512.3292|    Ward, Miss. Anna|
|        738|512.3292|Lesurer, Mr. Gust...|
|         89|   263.0|Fortune, Miss. Ma...|
|         28|   263.0|Fortune, Mr. Char...|
+-----------+--------+--------------------+
only showing top 5 rows



In [13]:
extra_info = [
    (1, 'VIP'),
    (2, 'Regular'),
    (3, 'VIP'),
    (4, 'Regular'),
    (5, 'VIP')
]
columns = ["PassengerId", "Status"]

extra_df = spark.createDataFrame(extra_info, columns)

extra_df.show(5)


+-----------+-------+
|PassengerId| Status|
+-----------+-------+
|          1|    VIP|
|          2|Regular|
|          3|    VIP|
|          4|Regular|
|          5|    VIP|
+-----------+-------+



In [14]:
# Join based on PassengerId
joined_df = titanic_df.join(extra_df, on="PassengerId", how="left")
joined_df.show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Status|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|    VIP|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|Regular|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|    VIP|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|Regular|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|    VIP|
|          6|   

In [15]:
joined_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Status|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|    VIP|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|Regular|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|    VIP|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|Regular|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|    VIP|
+-----------+---

In [16]:
# Convert to RDD
titanic_rdd = titanic_df.rdd.map(lambda row: (row.Pclass, (row.Fare, 1)))

# Define the aggregation logic
seqOp = (lambda acc, value: (acc[0] + value[0], acc[1] + value[1]))  # sum fare, increment count
combOp = (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]))  # combine sums and counts

# Apply aggregateByKey
fare_by_class = titanic_rdd.aggregateByKey((0, 0), seqOp, combOp)

# Compute the average fare by class
average_fare_by_class = fare_by_class.mapValues(lambda x: x[0] / x[1])
average_fare_by_class.collect()

[(3, 13.675550101832997), (1, 84.15468749999992), (2, 20.66218315217391)]

In [17]:
survivors_df = titanic_df.filter(titanic_df.Survived == 1)
survivors_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female|27.0|    0|    2|          347742|11.1333| null|       S|
|         10|       1|     2|Nasser, Mrs. Nich...|female|14.0|    1|    0|          237736|30.0708| null|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [18]:
from pyspark.sql.functions import col, sum

# Count the number of null values in each column
missing_values = titanic_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in titanic_df.columns])
missing_values.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [20]:
from pyspark.sql.functions import mean

# Calculate mean age
mean_age = titanic_df.select(mean(titanic_df['Age'])).collect()[0][0]

# Fill missing values in 'Age' with the mean
titanic_df = titanic_df.fillna({'Age': mean_age})

# Show the result
titanic_df.show(5)


+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [21]:
# Count the number of null values in each column
missing_values = titanic_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in titanic_df.columns])
missing_values.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [22]:
# Fill all missing values with a default constant (e.g., 0 or "Unknown")
titanic_df = titanic_df.fillna({"Cabin": "Unknown"})

In [23]:
# Count the number of null values in each column
missing_values = titanic_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in titanic_df.columns])
missing_values.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [24]:
# Drop rows where Age is null
titanic_df = titanic_df.dropna(subset=["Embarked"])
missing_values = titanic_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in titanic_df.columns])
missing_values.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [25]:
# Cache the dataframe
titanic_df.cache()

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [26]:
# Save DataFrame as CSV
cleaned_df.write.csv('cleaned_titanic_data.csv', header=True)